In [2]:
import pandas as pd
import json
import re

def clean_output(output):
    cleaned_output = output
    # Extract XML code from the output
    xml_code = re.search(r'```json(.*?)```', output, re.DOTALL)
    cleaned_output = ''
    if xml_code:
        cleaned_output = xml_code.group(1).strip()
    
    # Parse the JSON
    try:
        # cleaned_output = json.loads(cleaned_output)
        cleaned_output = eval(cleaned_output)
    except:
        return {}
    
    return cleaned_output

prompts = pd.read_json('fn1.7-test-prompts.jsonl', lines=True)

# predictions = pd.read_json('batch-gpt-4o-mini-0.0temperature-json-existing-cot-predictions-cleaned.jsonl', lines=True)
with open('batch-ft:gpt-4o-mini-2024-07-18:idir-lab:fn1-7-json-existing-smalltrain:Aau9E1GX-0.0temperature-json-existing-predictions-cleaned.jsonl') as f:
    predictions = f.readlines()

predictions = [json.loads(p.strip()) for p in predictions]

prompts['prediction'] = predictions
prompts['output'] = prompts.messages.apply(lambda x: clean_output(x[-1]['content']))
predictions = prompts[['output', 'prediction']]

In [4]:
tp = 0
near_miss = 0
fp = 0
fn = 0
tn = 0 # not really used because tags only have positive values, technically all other FEs are TN, but we don't really care about that (maybe we should?)

for label, pred, fes in predictions[['output', 'prediction', 'output']].values:

    # Get each predicted FE span from the prediction
    pred_tags = pred
    real_tags = fes

    # Check each predicted FE span
    for tag, content in pred_tags.items():
        if tag.capitalize() in real_tags:
            if content == real_tags[tag.capitalize()]:
                tp += 1
            else:
                # print(f'Near miss: {content} vs {real_tags[tag.capitalize()]}')
                fp += 1
                near_miss += 1
        else:
            fp += 1
    
    # Check each real FE span
    for tag, content in real_tags.items():
        if tag not in pred_tags:
            fn += 1
        elif content != pred_tags[tag]:
            fn += 1

# print(f'Perfect: {perfect} out of {len(predictions)}')
print(f'TP: {tp}')
print(f'Missed: {near_miss}')
print(f'FP: {fp}')
print(f'FN: {fn}')

print(f'Precision (ignoring near misses): {tp / (tp + fp - near_miss)}')
print(f'Precision: {tp / (tp + fp):.3f}')
print(f'Recall: {tp / (tp + fn):.3f}')
print(f'F1: {2 * tp / (2 * tp + fp + fn):.3f}')

print(f'Accuracy: {tp / (tp + fn + fp):.3f}')



TP: 7747
Missed: 1457
FP: 4201
FN: 3202
Precision (ignoring near misses): 0.7384424745019541
Precision: 0.648
Recall: 0.708
F1: 0.677
Accuracy: 0.511


Bad pipe message: %s [b'\xdb\x88\xbb6\x8c\xeb`?H\x94\xadn?\xaa\xf1\xae\x0b\\\x00\x01|\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00\x13\x00\x14\x00\x15\x00\x16\x00\x17\x00\x18\x00\x19\x00\x1a\x00\x1b\x00/\x000\x001\x002\x003\x004\x005\x006\x007\x008\x009\x00:\x00;\x00<\x00=\x00>\x00?\x00@\x00A\x00B\x00C\x00D\x00E\x00F\x00g\x00h\x00i\x00j\x00k\x00l\x00m\x00\x84\x00\x85\x00\x86\x00\x87\x00\x88\x00\x89\x00\x96\x00\x97\x00\x98\x00\x99\x00\x9a\x00\x9b\x00\x9c\x00\x9d\x00\x9e\x00\x9f\x00\xa0\x00\xa1\x00\xa2\x00\xa3\x00\xa4']
Bad pipe message: %s [b'\xdc\xde\x97V\xdcl\xd7M\x8b<\xd1 \x9b\xffUk\xc1\xe3\x00\x01|\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00\x13\x00\x14\x00\x15\x00\x16\x00\x17\x00\x18\x00\x19\x00\x1a\x00\x1b\x00/\x000\x001\x002\x003\x004\x005\x006\x007\x008\x009\x00:\x00;\x00